# **Factors of Fraud**

## Objectives
* Answer business requirement 1:
    * ACB would like to understand the patterns in the transaction data to better understand the most relevant variables correlated to a fraudulent transaction.*   

## Inputs

* outputs/datasets/collection/card_transactions.csv

## Outputs

* Generate code and visualisations that fulfil business requirement 1, above.